# Manipulating models

## Preparation

In [18]:
from cobra.io import read_sbml_model
model = read_sbml_model('data/e_coli_core.xml.gz')

## Making temporary changes to the model (5 + 2)

Usually one relies on making copies if objects need to be changed but the original state needs to be retained and that's we what we did previously with the `model_copy = model.copy()`. Unfortunately, metabolic models can be enormous and copying those in memory can take quite long.

In [2]:
%%time
copy_of_model = model.copy()

CPU times: user 36.4 ms, sys: 7.51 ms, total: 43.9 ms
Wall time: 62.9 ms


Yes, even milliseconds add up pretty quickly if you need to run many simulation (e.g. if you need to knock out every single gene individually in the model to check if it is essential or not).

In [3]:
%%time
## avoid writing code like this...
for gene in model.genes:
    mutant = model.copy()
    mutant.genes.get_by_id(gene.id).knock_out()

CPU times: user 3.02 s, sys: 211 ms, total: 3.23 s
Wall time: 4.42 s


For that reason cobrapy provides a mechanism that is less time consuming. Almost all methods that make changes to the mdoel such as knocking-out genes, reactions, adding or removing metabolites, reactions etc can be automatically reverted upon exit from a python context. How this works is probably best understood by looking at an example.

In [4]:
%%time
## instead do this; no copying!
with model:
    for gene in model.genes:
        gene.knock_out()

CPU times: user 21.6 ms, sys: 1.3 ms, total: 22.9 ms
Wall time: 22 ms


Here, the `with model` statements starts the context and changes done to the model one indentation level to the right, are automatically recorded. When that block finishes, the context manager is requested to roll-back all changes leaving the model looking exactly as it did before all the changes.

Changing flux bounds can as indicated also be done reversibly. For example let's set the lower and upper bound of phosphoglycerate kinase to 0 (effectively knocking out the reaction).

In [5]:
with model:
    model.reactions.PGK.bounds = 0, 0
    print("PGK's bounds inside the with statement")
    print(model.reactions.PGK.lower_bound, model.reactions.PGK.bounds)
    print('Mutant growth rate: ', model.optimize().objective_value)
print("PGK's bounds outside the with statement")
print(model.reactions.PGK.bounds)

PGK's bounds inside the with statement
0 (0, 0)
Mutant growth rate:  0.0
PGK's bounds outside the with statement
(-1000.0, 1000.0)


### Exercises

Convert the cell below to code and fill in the blanks

## Changing the medium (2 + 3)

One can access the medium condition using `model.medium`. The indicated bound is the effective upper uptake bound. 

In [6]:
model.medium

{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0}

Changing the carbon source in the medium can be achieved by adjusting the flux bounds of the respective exchange reactions appropriately. For example, the following code block removes glucose from the medium and adds succinate.

In [7]:
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 0
    medium['EX_succ_e'] = 10
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['BIOMASS_Ecoli_core_w_GAM'])

0.397563015428


Changing the carbon source to succinate led to a significant drop in growth rate.

### Exercise

Convert the cell below to code and fill in the blanks

## Adding metabolites, reactions and pathways (10 + 10)

In [19]:
from cobra import Reaction, Metabolite

Ok, let's create a new reactions.

In [20]:
new_reaction = Reaction('alchemy')

This reaction is going to convert water into gold. First we need to create a new metabolite, since gold is not yet part of *E. coli's* native metabolism.

In [21]:
gold = Metabolite(id='gold_c', compartment='c', name='GOLD')

Now, we're going to specify the reaction's stoichiometry.

In [22]:
new_reaction.add_metabolites({model.metabolites.h2o_c: -1, gold: 1})

Printing the reaction reveals that the reaction indeed converts water into gold.

In [23]:
print(new_reaction.build_reaction_string())

h2o_c --> gold_c


Now, let's add the new reaction to the model.

In [24]:
model.add_reactions([new_reaction])

Quickly check that the reaction was indeed added to the model.

In [25]:
model.reactions.alchemy

Reaction identifier,alchemy
Name,
Memory address,0x0119bc0320
Stoichiometry,h2o_c --> gold_c H2O --> GOLD
GPR,
Lower bound,0.0
Upper bound,1000.0


Let's produce some gold then!

In [26]:
model.objective = model.reactions.alchemy
model.optimize().objective_value

0.0

:-(

What happened? Forgot to add an exchange reaction so that gold can leave the system.

In [27]:
model.add_boundary(model.metabolites.gold_c, type='demand')

Reaction identifier,DM_gold_c
Name,GOLD demand
Memory address,0x0119ba8438
Stoichiometry,gold_c --> GOLD -->
GPR,
Lower bound,0
Upper bound,1000.0


In [28]:
model.objective = model.reactions.alchemy
model.optimize().objective_value

1000.0

Yes, much better!

### Exercise


Convert the cell below to code and fill in the blanks. Add a magic pathway that converts glucose directly to atp. Does this lead to an increase in growth rate?

Time left? Add a pathway of your choice and optimize for flux through it!